In [1]:
MODEL_PATH = 'location_model.pkl'
LOCATIONS_PATH = 'locations_lat_long.csv'
INPUT_TRANSFORMER_PATH = 'input_transformer.pkl'
OUTPUT_TRANSFORMER_PATH = 'output_transformer.pkl'

In [26]:
import re
import pandas as pd
import pickle 
class ObjectsManagement:
    """read and write objects on given path.
    works by pickel."""
    def read_object(path):
        try:
            with open(path,'rb') as file:
                object_ = pickle.load(file)
        except Exception as ex:
            print('Some error in reading from file:',path)
            raise ex
        return object_
    
    def write_object(path):
        pass

class InputTransformer:
    def __init__(self,**kwargs):
        """init(**kwargs)---> parameters:
        input_transformer_path: path of text transformer, required.
        preprocess_ptn: regular expression for text preprocessing, optional, default: r'[^A-Za-zأ-ي0-9 ]'"""
        try:
            self.preprocess_ptn = kwargs.get('preprocess_ptn',r'[^A-Za-zأ-ي0-9 ]')
            self.input_transformer = ObjectsManagement.read_object(kwargs['input_transformer_path']) 
        except Exception as ex:
            print('Some Error in init InputPreprocess',ex)
            raise ex
            
    def transform(self,location_string,**kwargs):
        try:
            preprocessed_str = []
            for ls in location_string:
                preprocessed_str.append(re.sub(self.preprocess_ptn,'',ls))
            input_vector = self.input_transformer.transform(preprocessed_str)
            return input_vector
        except Exception as ex:
            print('Some Error in InputPreprocess->transform:',ex)
            raise ex

class LocationNormalizer:
    """this class to normalize name of Location."""
    def __init__(self,**kwargs):
        """init(**kwargs)---> **kwargs parameters:
        input_transformer_path: path of text transformer.
        output_transformer_path: path of labels transformer
        model_path: path of model."""
        try:
            self.input_transformer = InputTransformer(input_transformer_path= kwargs['input_transformer_path'])
            
            self.output_transformer = ObjectsManagement.read_object(kwargs['output_transformer_path'])
            
            self.model = ObjectsManagement.read_object(kwargs['model_path'])
        except Exception as ex:
            print('Some error in LocationNormalizer->init:',ex)
            
    def get_normalized_location(self,location_string,**kwargs):
        try:
            location_vector = self.input_transformer.transform(location_string)
            prediction = self.model.predict(location_vector)
            normalized_location = self.output_transformer.inverse_transform(prediction)
            return normalized_location
        except Exception as ex:
            print('Some error in LocationNormalizer->get_location:',ex)
            raise ex
        
class LatLongPredictor:
    """this class to get latitude and longitude of Location"""
    def __init__(self,**kwargs):
        """init(**kwargs)---> **kwargs parameters:
        locations_path: file of locations.
        locations file must contain columns: label,latitude,longitude"""
        try:
            self.locations = pd.read_csv(kwargs['locations_path'])
        except Exception as ex:
            print('Some error in LatLongPredictor->init:',ex)
            raise ex
    def get_lat_long(self,locations,**kwargs):
        try:
            df = pd.DataFrame(columns = ['label','latitude','longitude'])
            for location in locations:
                df= df.append(self.locations[self.locations['label']==location],ignore_index=True)
            return df
        except Exception as ex:
            print('Some error in LatLongPredictor->get_lat_long:',ex)
            raise ex
    
class Locator:
    """this class to get user latitude and longitude.
    first normalize his location then get latitude and longitude"""
    def __init__(self,**kwargs):
        """init(**kwargs)---> **kwargs parameters:
        input_transformer_path: path of text transformer.
        output_transformer_path: path of labels transformer
        model_path: path of model
        locations_path: file of locations.
        """
        try:
            self.location_normalizer = LocationNormalizer(model_path= kwargs['model_path'],output_transformer_path= kwargs['output_transformer_path'],
                                                          input_transformer_path= kwargs['input_transformer_path'])
            
            self.lat_long_predictor = LatLongPredictor(locations_path= kwargs['locations_path'])
        except Exception as ex:
            print('Some Error in Locator->init:',ex)
            
    def get_locate(self,location_str,**kwargs):
        """get_locate(location_str,**kwargs)--->
        this function returns the latitude-longitude of passed location.
        parameters: 
        location_str: the location as user entered."""
        try:
            normalized_str = self.location_normalizer.get_normalized_location(location_str)
            return self.lat_long_predictor.get_lat_long(normalized_str)
        except Exception as ex:
            print('Some Error in Locator->get_locate:',ex)

In [27]:
locator = Locator(input_transformer_path=INPUT_TRANSFORMER_PATH,output_transformer_path=OUTPUT_TRANSFORMER_PATH,
                  model_path=MODEL_PATH,locations_path=LOCATIONS_PATH)

In [42]:
def user_location_lang_lat():
    df = pd.read_csv('full_order.csv')
    result=df[['user_id',"address_"]]
    r=locator.get_locate(result['address_'][:10])
    r['user_id'] = result['user_id']
    r[['user_id','latitude','longitude']].to_csv("users_locations_lat_long.csv",index=False)
user_location_lang_lat()

In [47]:
def rest_location_lang_lat():
    df = pd.read_csv('full_order.csv')
    result=df[['restaurant_id',"adress","a_name"]]


    r=locator.get_locate(result['adress'])
    r['id'] = result['restaurant_id']
    r['name'] = result['a_name']
    
    r[['id','name','latitude','longitude']].to_csv("rests_loactions_lat_long.csv",index=False)
rest_location_lang_lat()